In [5]:
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import ast


import json

import numpy as np

from datetime import timedelta, date


import re

%pip install lxml
%pip install requests
%pip install sodapy

from lxml import html
import requests

import os

import glob

from sodapy import Socrata


You should consider upgrading via the 'c:\Users\José Luis Adriano\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\José Luis Adriano\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\José Luis Adriano\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


### <b>Version 1 Data</b>

#### Get limit (needs debugging)

In [6]:
#NOTE: ideally change to webpage with the data
page = requests.get('https://www.dallasopendata.com/Public-Safety/Police-Incidents/qv6i-rri7')
webpage = html.fromstring(page.content)

#NOTE: this does not work right now
lim = webpage.xpath('/html/body/main/div/div[1]/div/div[2]/forge-view-switcher/forge-view[1]/div/div/div/div[1]/section[1]/div/div/dl/div[1]/dd')
print(lim)

[]


#### Read in data

In [7]:
#Define Socrata API Client
client = Socrata("www.dallasopendata.com", None)

#NOTE: Change to the limit we defined earlier
lim = 1200000

#Read in data and convert to DataFrame
results = client.get("qv6i-rri7", limit=lim)

df = pd.DataFrame.from_records(results)

    # lim += 100000

print(len(df))

#Create empty lists of addresses, latitudes and longitudes to convert to columns later.
lats = []
longs = []
adds = []

#In the underlying raw date we have a column called 'Location1' where each cell contains a street address, latitude and Longitude. Lets extract those and add them to the list.
for i in range(0, len(df)):
    #If the cell isn't empty
    try:
        latitude = df['geocoded_column'][i]['latitude']
    except:
            latitude = np.nan
    try:
        longitude = df['geocoded_column'][i]['longitude']
    except:
        longitude = np.nan
    try:
        address = ''
        for val in ast.literal_eval(df['geocoded_column'][i]['human_address']).values():
             address = address + ' ' + val.strip()
    except:
         address = ''
        
    #Add the lat, long and address to the corresponding list.
    lats.append(latitude)
    longs.append(longitude)
    adds.append(address)

# #Create new columns from the populated lists.
df['latitude'] = lats
df['longitude'] = longs
df['address'] = adds 

df['date1'] = pd.to_datetime(df['date1'], errors='coerce')

### <b>Version 2 Data</b>

In [ ]:
#The data are stored in Excel spreadsheets with one year's worth of incident data. Let's get a list of these files, so we can combine them. 
path = 'data/raw/dpd/'
files = os.listdir(path)
files = [i for i in files if 'Open_Data' in i]

#Create an empty dataframe to add our data to.
df1 = pd.DataFrame()

#Go through each file.
for file in files:
    #Most of our files just use the default 'Sheet1' for their sheet name. But in 2018, 2019, 2020 and 2022 the sheet name is the year. 
    if file not in ['Open_Data_2018.xlsx', 'Open_Data_2019.xlsx', 'Open_Data_2020.xlsx', 'Open_Data_2022.xlsx']:
        #If the data is not one of those years, then just use Sheet1 as the sheet name. 
        dfx = pd.read_excel(f'{path}{file}', sheet_name = 'Sheet1')

        #Try to extract the year from the filename and assign that value to a column.
        try:
            dfx['year'] = int(file[-9:-5])
        
        #If the year isn't in that position on the string, it's our half year of data from 2014.
        except:
            dfx['year'] = 2014 
    
    #If it is one of those years, extract it from the filename, use that as the sheet name an assign that value to a column.
    else:
       dfx = pd.read_excel(f'{path}{file}', sheet_name = file[-9:-5])
       dfx['year'] = int(file[-9:-5])
    
    #Add that data to our empty df
    df1 = pd.concat([df1, dfx])

#Each of the files we add to the empty df will start from 0, so we need to reset the index so that the numbers are unique for each row.
df1 = df1.reset_index(drop=True)
df1.columns = [i.lower().strip().replace(' ', '_').replace('/', '').replace('(', '').replace(')', '').replace('#', '').replace('-', '') for i in list(df1.columns)]

df1['point_x'] = pd.to_numeric(df1['point_x'], errors='coerce')
df1['point_y'] = pd.to_numeric(df1['point_y'], errors='coerce')

#convert incidents to a spatial file
gdf = gpd.GeoDataFrame(df1, geometry = gpd.points_from_xy(df1['point_x'], df1['point_y']))

#project using DPDs weird CRS
gdf = gdf.set_crs("EPSG:2276")

#re-project using a normal CRS
gdf = gdf.to_crs("EPSG:4326")

gdf['latitude'] = gdf['geometry'].y
gdf['longitude'] = gdf['geometry'].x

df1 = gdf.copy()

### <b>Version 3 Data</b>

In [ ]:
#read in incidents data
df2a = pd.read_excel('data/raw/dpd/Data From Jan 1, 2017 to Apr 13, 2023.xlsx', sheet_name='From Jan 1, 2017-Apr 13, 2023')
df2b = pd.read_excel('data/raw/dpd/Data of 2014 to 2016.xls', sheet_name=' Murder Data 2014-2016 (01%)')
df2c = pd.read_excel('data/raw/dpd/Data of 2014 to 2016.xls', sheet_name='(03%)  Robbery Data 2014-2016')
df2d = pd.read_excel('data/raw/dpd/Data of 2014 to 2016.xls', sheet_name='(04%) Agg Assault 2014-2016')

In [ ]:
df2 = pd.concat([df2a, df2b, df2c, df2d])
df2.columns = [i.lower().strip().replace(' ', '_').replace('/', '').replace('(', '').replace(')', '').replace('#', '').replace('-', '') for i in list(df2.columns)]

In [ ]:
df2['point_x'] = pd.to_numeric(df2['point_x'], errors='coerce')
df2['point_y'] = pd.to_numeric(df2['point_y'], errors='coerce')

#convert incidents to a spatial file
gdf = gpd.GeoDataFrame(df2, geometry = gpd.points_from_xy(df2['point_x'], df2['point_y']))
#project using DPDs weird CRS
gdf = gdf.set_crs("EPSG:2276")

#re-project using a normal CRS
gdf = gdf.to_crs("EPSG:4326")

gdf['latitude'] = gdf['geometry'].y
gdf['longitude'] = gdf['geometry'].x

df2 = gdf.copy()

### <b>Version 4 Data</b>

In [ ]:
#read in latest batch of incidents
df3 = pd.read_excel('data/raw/dpd/D009766-032123__Data_from_2014-to_present.xlsx', sheet_name='Sheet1')

In [ ]:
#Lowercase all of our columns then, strip whitespace, replace spaces with underscores and get rid of unnecssary symbols like slashes, dashes, parentheses and number signs.
df3.columns = [i.lower().strip().replace(' ', '_').replace('/', '').replace('(', '').replace(')', '').replace('#', '').replace('-', '') for i in list(df3.columns)]

In [ ]:
#convert incidents to a spatial file
gdf = gpd.GeoDataFrame(df3, geometry = gpd.points_from_xy(df3['point_x'], df3['point_y']))

#project using DPDs weird CRS
gdf = gdf.set_crs("EPSG:2276")

#re-project using a normal CRS
gdf = gdf.to_crs("EPSG:4326")

In [ ]:
gdf['latitude'] = gdf['geometry'].y
gdf['longitude'] = gdf['geometry'].x
df3 = gdf.copy()

### <b>Combining the Data<b>

In [ ]:
df['version'] = 1
df1['version'] = 2
df2['version'] = 3
df3['version'] = 4

In [ ]:
df = df.rename(columns = {'zip_code': 'zipcode', 'x_coordinate': 'point_x', 'y_cordinate':'point_y' })
print('not in df1: ', [i for i in list(df.columns) if i not in list(df1.columns)])
print('not in df2: ', [i for i in list(df.columns) if i not in list(df2.columns)])
print('not in df3: ', [i for i in list(df.columns) if i not in list(df3.columns)])

In [ ]:
df1 = df1.rename(columns = {'apt#':'apt'})
print('not in df: ', [i for i in list(df1.columns) if i not in list(df.columns)])
print('not in df2: ', [i for i in list(df1.columns) if i not in list(df2.columns)])
print('not in df3: ', [i for i in list(df1.columns) if i not in list(df3.columns)])

In [ ]:
df2 = df2.rename(columns = {'apt#':'apt'})
print('not in df: ', [i for i in list(df2.columns) if i not in list(df.columns)])
print('not in df1: ', [i for i in list(df2.columns) if i not in list(df1.columns)])
print('not in df3: ', [i for i in list(df2.columns) if i not in list(df3.columns)])

In [ ]:
print('not in df: ', [i for i in list(df3.columns) if i not in list(df.columns)])
print('not in df1: ', [i for i in list(df3.columns) if i not in list(df1.columns)])
print('not in df2: ', [i for i in list(df3.columns) if i not in list(df2.columns)])

In [ ]:
df = pd.concat([df, df1, df2, df3], ignore_index=True)

### <b>Filter to Just Murders</b>

In [ ]:
df['date1'] = pd.to_datetime(df['date1'], errors='coerce')

df = (df
      .loc[((df['nibrs_code'] == '09A') | 
            (df['ucrcode'].isin([110, 120])))]
        .sort_values('date1', ascending=False)
        .drop_duplicates(subset = ['incidentnum'], keep='first')
        .reset_index(drop=True)
        .copy())

In [ ]:
df.to_csv('data/created/dpd_murders_data.csv', index=False)